In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4
!pip install wtforms==2.3.3
# Инициализация базы данных
!airflow db init

In [ ]:
# Создадим необходимые папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

In [ ]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

In [ ]:
# Создадим пользователя Airflow
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

In [ ]:
# Запуск шедулера
!airflow scheduler -D

In [ ]:
!pip install pyngrok
!ngrok authtoken 2LGqlyeTG6YSlrdLQHg1o4m6243_7XFTFKwhccWtFhfAgL7o2 #  https://dashboard.ngrok.com/get-started/setup 
# https://dashboard.ngrok.com/endpoints/status
!nohup ngrok http -log=stdout 18273 > /dev/null &

In [ ]:
# Решение на разработку
from airflow import DAG
from datetime import timedelta
from datetime import date
from airflow.utils.dates import days_ago
from airflow.operators.python import PythonOperator
import pandas as pd
from sqlalchemy import create_engine
from  sqlalchemy.exc import OperationalError
CONN = create_engine('sqlite:///test_task_db.db', echo=False)

try:
    CONN.execute("CREATE TABLE JOIN_DATA (currency TEXT NOT NULL,value INTEGER,date TEXT NOT NULL,rates TEXT NOT NULL)")
except OperationalError:
    pass  

def extract_currency(date):
  import requests
  url = 'https://api.exchangerate.host/timeseries?start_date='+str(date)+'&end_date='+str(date)+'&base=EUR&symbols=USD&format=json'
  response = requests.get(url)
  data = response.json()
  extract_key = list(data['rates'][str(date)].keys())[0]
  extract_value = list(data['rates'][str(date)].values())[0]
  data['rates'] = str(extract_key) + ' : ' + str(extract_value)
  data['currency'] = data.pop('base')
  data['date'] = data.pop('start_date')
  result = {key: [data[key]] for key in data.keys() & {'currency', 'rates', 'date'}}
  pd.DataFrame(result).to_sql('currency', con=CONN)

def load_db():
  CONN.execute('INSERT INTO JOIN_DATA(currency, date, rates) SELECT currency, date, rates FROM currency')
  CONN.execute('DROP TABLE currency')

with DAG('dag', schedule_interval=timedelta(days=1), start_date=days_ago(1)) as dag:
  t1 = PythonOperator(task_id='load_from_source',
                      python_callable=extract_currency,
                      op_kwargs={'date': '2021-01-01'}
  )
  t2 = PythonOperator(task_id='db_load', 
                      python_callable=load_db
  )
  t1 >> t2
  import airflow
from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.python_operator import PythonOperator

args = {'owner': 'airflow', 'start_date': days_ago(1), 'provide_context': True }
dag = DAG('dag',schedule_interval='@once', default_args=args)

value_2 = {'a': 'b'}

def push(**kwargs):
    kwargs['ti'].xcom_push(key='key', value=value_2)

def pull(**kwargs):
    print(kwargs['ti'].xcom_pull(key='key', task_ids='push'))

push1 = PythonOperator(
    task_id='push', dag=dag, python_callable=push)
pull1 = PythonOperator(
    task_id='pull', dag=dag, python_callable=pull)

push1 >> pull1